In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#every object will have unique id

#Bytetrack - if some object disappear and appear it will assign new id each time

#Bot sort - if some object disappear and appear it will always assign same id each time(slower)

In [43]:
!pip install ultralytics
!pip install roboflow

In [44]:
import ultralytics
ultralytics.checks()

Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6634.4/8062.4 GB disk)


In [45]:
!nvidia-smi

Fri Jan 30 06:50:53 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             33W /  250W |    1155MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#train

# people detection for tracking

In [46]:
from roboflow import Roboflow
rf = Roboflow(api_key="nXRFu1swQzbMlfRMolSH")
project = rf.workspace("chris-kydks").project("people-detection-2csbw")
version = project.version(11)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...


In [47]:
#config
import yaml

yaml_path = "/kaggle/working/People-Detection-11/data.yaml"   # or config.yml

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

print(data)


data["path"] = "/kaggle/working/People-Detection-11"

# optional but recommended
data["train"] = "/kaggle/working/People-Detection-11/train/images"
data["val"] = "/kaggle/working/People-Detection-11/valid/images"
data["test"]= "/kaggle/working/People-Detection-11/test/images"

with open(yaml_path, "w") as f:
    yaml.dump(data, f, sort_keys=False)

print("YAML updated for Kaggle ✅")

{'train': '/kaggle/working/People-Detection-11/train/images', 'val': '/kaggle/working/People-Detection-11/valid/images', 'test': '/kaggle/working/People-Detection-11/test/images', 'nc': 1, 'names': ['Person'], 'roboflow': {'workspace': 'chris-kydks', 'project': 'people-detection-2csbw', 'version': 11, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/chris-kydks/people-detection-2csbw/dataset/11'}, 'path': '/kaggle/working/People-Detection-11'}
YAML updated for Kaggle ✅


In [48]:
!cat /kaggle/working/People-Detection-11/data.yaml

train: /kaggle/working/People-Detection-11/train/images
val: /kaggle/working/People-Detection-11/valid/images
test: /kaggle/working/People-Detection-11/test/images
nc: 1
names:
- Person
roboflow:
  workspace: chris-kydks
  project: people-detection-2csbw
  version: 11
  license: CC BY 4.0
  url: https://universe.roboflow.com/chris-kydks/people-detection-2csbw/dataset/11
path: /kaggle/working/People-Detection-11


In [49]:
!gdown "https://drive.google.com/uc?id=1Nyl18zr7zOJHh8uHtSic3dsJSY4LI_t-&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1Nyl18zr7zOJHh8uHtSic3dsJSY4LI_t-&confirm=t
To: /kaggle/working/PPE_Part1.mp4
100%|██████████████████████████████████████| 14.2M/14.2M [00:00<00:00, 80.0MB/s]


In [50]:
!gdown "https://drive.google.com/uc?id=1SxcrDanPAr-PZfJjWBxoa2t8lUSM2dZn&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1SxcrDanPAr-PZfJjWBxoa2t8lUSM2dZn&confirm=t
To: /kaggle/working/PPE_Part2.mp4
100%|███████████████████████████████████████| 11.1M/11.1M [00:00<00:00, 213MB/s]


# detecting using trained model

In [51]:
!yolo task=detect model=train data=/kaggle/working/People-Detection-11/data.yaml model="yolo11n.pt" epochs=10 imgsz=640

WARNING ⚠️ 'mode' argument is missing. Valid modes are ['export', 'benchmark', 'val', 'track', 'train', 'predict']. Using default 'mode=train'.
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/People-Detection-11/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11

In [52]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/runs/detect/train3/weights/best.pt")

results = model.track(source="/kaggle/working/PPE_Part1.mp4", persist=True, save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/310) /kaggle/working/PPE_Part1.mp4: 384x640 3 Persons, 11.6ms
video 1/1 (frame 2/310) /kaggle/working/PPE_Part1.mp4: 384x640 3 Persons, 10.0ms
video 1/1 (frame 3/310) /kaggle/working/PPE_Part1.mp4: 384x640 3 Persons, 9.7ms
video 1/1 (frame 4/310) /kaggle/working/PPE_Part1.mp4: 384x640 3 Persons, 9.7ms
video 1/1 (frame 5/310) /kaggle/working/PPE_Part1.mp4: 384x640 3 Persons, 9.5ms
video 1/1 (frame 6/310) /kaggle/working/PPE_Part1.mp4: 3

# Detecting and  using yolo11m model without training

In [53]:
#detecting
from ultralytics import YOLO

# Load a model
model1 = YOLO("yolo11m.pt")

# Perform object detection on video
results = model1("/kaggle/working/PPE_Part1.mp4", save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/310) /kaggle/working/PPE_Part1.mp4: 384x640 7 persons, 1 bench, 1 tv, 14.0ms
video 1/1 (frame 2/310) /kaggle/working/PPE_Part1.mp4: 384x640 7 persons, 1 tv, 13.2ms
video 1/1 (frame 3/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 1 tv, 14.7ms
video 1/1 (frame 4/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 1 tv, 13.2ms
video 1/1 (frame 5/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 1 tv, 13.2ms
video 1/1 (fra

In [54]:
#only class 0 prediction ( people)
results = model1("/kaggle/working/PPE_Part1.mp4",classes=0, save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/310) /kaggle/working/PPE_Part1.mp4: 384x640 7 persons, 14.5ms
video 1/1 (frame 2/310) /kaggle/working/PPE_Part1.mp4: 384x640 7 persons, 15.2ms
video 1/1 (frame 3/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 15.7ms
video 1/1 (frame 4/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 15.7ms
video 1/1 (frame 5/310) /kaggle/working/PPE_Part1.mp4: 384x640 6 persons, 14.7ms
video 1/1 (frame 6/310) /kaggle/working/PPE_Part1.mp4

In [55]:
#Bot Sort tracking
results = model1.track(source="/kaggle/working/PPE_Part2.mp4", persist=True, save=True)


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/267) /kaggle/working/PPE_Part2.mp4: 384x640 1 person, 14.5ms
video 1/1 (frame 2/267) /kaggle/working/PPE_Part2.mp4: 384x640 1 person, 14.6ms
video 1/1 (frame 3/267) /kaggle/working/PPE_Part2.mp4: 384x640 2 persons, 15.0ms
video 1/1 (frame 4/267) /kaggle/working/PPE_Part2.mp4: 384x640 2 persons, 15.0ms
video 1/1 (frame 5/267) /kaggle/working/PPE_Part2.mp4: 384x640 2 persons, 14.5ms
video 1/1 (frame 6/267) /kaggle/working/PPE_Part2.mp4: 